In [ ]:
# LEFT TO DO:
 
'''
- Integrate this code into the main data_collection.ipynb, specifically, get the appropriate parts under the generate_stats function. 
- Work on outputting the stats correctly.
- F6 as in Darmon's work?

'''


In [46]:
import re
import itertools
import pandas as pd
from statistics import mean
from string import punctuation
from itertools import chain

In [3]:
# Helper function which splits text input into individual sentances. Used in generation of f4.

acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
alphabets= "([A-Za-z])"
digits = "([0-9])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text) 
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [23]:
# Input, our array of the ten most recent comments from a single Redditor.
comment_list = ["35, I have a pretty good job but I don't love it. I'm still trying to figure out what I want to be when I grow up. Situation could be a lot worse but I'm at a point where I want to do something I enjoy.", "My family (live with my real mom and step dad) act like me being outed by older sister never happened. And on top of all of this I'm struggling trying to study for my math ged which I failed six times with being 1 point shy from passing. If I fail it again, its another two months of waiting to retake it.", "I'm really not sure how i'm supposed to feel anymore knowing that most of my life I was just wasting my time.", "I'm an IT apprentice and was about a month away from finishing my level 4 course. My training provider has just closed down due to a fraud investigation so now im in a position of trying to find a provider for my specific course so I can finish. And there's no guarantee my years work can be transferred so I may not even be able to get my certificate without another years of work.", "29 and I don't like being a mum", "I was going to complain about being a 24yo male who's single and doesn't want to be but some of these responses make mine a drop in the bucket compared to the waters some of you face. Best of luck to all of you.", "Split up with my long term so of 5 years and decided to go for a complete career change. Going to pack in this life and go travel for a bit. Experience the world and figure out what it is I want.", "I'm 30, diagnosed last year with ADHS, failed university this year, just needed to write my thesis, no intimacy with my girlfriend of 2 years anymore. Yeah Life!", "32, alcoholic, narcissistic, sociopath, broke, in debt, and i feel eventually my finances will magically fix themselves. I’m also a cocky piece of shit who doesn’t take advice or think there’s anything wrong with me. I truthfully think I’m ok. I need help. So much.", "My dad has lung cancer and is refusing treatment, which is fine. He's a 68 year old man who loves to drink and smoke and has accepted the hand he was dealt. He doesn't have to go through treatment if he doesn't want too."]

In [68]:
# This cell deals with reformatting of the comment-array input.

# Concatinates all the comments into one sample of the Redditor's writing (string object).
writing_sample = ' '.join(comment_list)

# Now, we need to check for instances of elipses and assign them to a new symbol--less confusion with periods, easier analysis.
# The tilde will represent an elipse from here on out.
writing_sample = writing_sample.replace('...', '~')


# We will loop over every character in the string and extract the punctuation, appending each mark to a list.
# We will keep this punctuation sequence in both list and string form for further analysis.
marks = ['.',',',':',';','?','!','-','(',')','"',"'",'~']
marks_str = ''.join(map(str, marks))

punct_list = []

for character in writing_sample:
    if character in marks:
        punct_list.append(character)
      
punct_str = ''.join(punct_list)

# Now, we want to get our input in the following format..[2, '!', 3, '.'].
split_by_punc = (x.split() for x in re.split('[' + punctuation + ']', writing_sample))
wrds_btwn = list(map(len, split_by_punc))
woven = list(chain.from_iterable(zip(wrds_btwn, punct_list)))

In [56]:
# Generation of f1, relative frequency of each punctuation mark of interest.
num_period = punct_str.count('.')
num_comma = punct_str.count(',')
num_colon = punct_str.count(':')
num_semicol = punct_str.count(';')
num_question = punct_str.count('?')
num_exclaim = punct_str.count('!')
num_dash = punct_str.count('-')
num_left_paren = punct_str.count('(')
num_right_paren = punct_str.count(')')
num_sing_quote = punct_str.count("'")
num_doub_quote = punct_str.count('"')
num_elipse = punct_str.count('~')

mark_counts = [num_period, num_comma, num_colon, num_semicol, num_question, num_exclaim, num_dash, num_left_paren, num_right_paren, num_sing_quote, num_doub_quote, num_elipse] 
f1 = [x / len(punct_list) for x in mark_counts]

In [70]:
# Generation of f2 and f3, conditional/joint probability of observing two punctuation marks in succession.
f2 = {}
f3 = {}
successive_pairs = []
combos = list(itertools.product(marks_str, marks_str))

for i in combos:
    successive_pairs.append(''.join(i))

for pair in successive_pairs:
    numerator = punct_str.count(pair)
    denominator = punct_str.count(pair[0])
    
    if numerator != 0:
        f2['Conditional Prob' + ' ' + pair + ' '] = (numerator/denominator)
        f3['Joint Prob' + ' ' + pair + ' '] = (numerator/len(punct_list))
    else:
        f2['Conditional Prob' + ' ' + pair + ' '] = 0
        f3['Joint Prob' + ' ' + pair + ' '] = 0

In [97]:
# Generation of f4, redditor's average sentence length in words.
sent_lens = []
regex = "\w+('\w+)?(?<!('s))"
sentences = split_into_sentences(writing_sample)

for sentence in sentences:
    sent_lens.append(len(re.findall(regex, sentence)))
    
f4 = mean(sent_lens)

# f4 as a probability distribution of sentence length--as it is in Darmon's work.
# Cap sentence length at 199 words.
F4 = [0]*200

for i in sent_lens:
    F4[i] += 1
    
F4 = [x / len(sent_lens) for x in F4]

In [101]:
# Generation of f5, redditor's average number of words between successive punctuation marks.
f5 = mean(wrds_btwn)

# f5 as a probability distribution of number of words between successive marks--as it is in Darmon's work.
# Cap number of in-between words at 39.
F5 = [0]*40

for i in wrds_btwn:
    F5[i] += 1
    
F5 = [x / len(wrds_btwn) for x in F5]

In [85]:
# Generation of f6, ratio of punctuation to words.
total_punc = len(punct_list)
total_words = len(re.findall(regex, writing_sample))

f6 = total_punc/total_words